# Redis as Vector Database to provide primary content to prompts

## Intro

Redis provides with [Redis Stack](https://redis.io/docs/stack/about/) an extension that adds modern data models and processing engines to the well know Redis in-memory data structure store. 

By extending Redis with Redis Stack it can be used as Vector Database to store embeddings (vectors) and to perform semantic search on the embeddings.

### Step 1: Azure Environment

This [Azure CLI script](../CreateEnv/CreateEnv.azcli) creates: 
- an Azure Open AI instance
- deploys `text-embedding-ada-002` to calculate embeddings
- starts a Redis Stack container on Azure Container Instances

The script provides necessary credentials (API key) and endpoint information (OpenAI, Redis) and stores them in environment variables.

```azurecli
$ENV:AZURE_OPENAI_ENDPOINT = $csEndpoint
$ENV:AZURE_OPENAI_API_KEY = $csApiKey
$ENV:AZURE_OPENAI_DEPLOYMENTNAME = $modelDeploymentName
$ENV:REDIS_PUBLIC_IP = $redisPublicIp
```

### Step 2: Arbitrary facts

Let's define some demo facts to be transformed into embeddings and stored in Redis.

In [2]:
//Some arbitrary facts
Dictionary<string, string> facts = new Dictionary<string, string>(); 
facts.Add("Mercury", "Mercury is the smallest planet in our solar system and the closest one to the Sun. It is approximately one-third the size of Earth.");
facts.Add("Saturn", "Saturn is a gas giant with a prominent system of rings, making it one of the most visually striking planets in our solar system.");
facts.Add("Germany", "Germany is a highly developed nation in central Europe, known for its rich history, technological advancements, and vibrant cultural scene.");

#!share --from c# facts --as facts

### Step 3: Connectivity

Definition of:
- Azure OpenAI client
- Connection to Redis Stack

In [6]:
#r "nuget: Azure.AI.OpenAi, 1.0.0-beta.5"
#r "nuget: NRedisStack, 0.6.1"

using Azure; 
using Azure.AI.OpenAI;
using StackExchange.Redis;
using NRedisStack;
using NRedisStack.RedisStackCommands;
using NRedisStack.Search;
using NRedisStack.Search.Literals.Enums;
using static NRedisStack.Search.Schema;

string apiKey = "<<your API key>>"; 
Uri apiEndpoint = new Uri("https://<<your API endpoint/"); 
string redisUri = "<<your Redis Stack IP>>"; 

//Create OpenAI Client
AzureKeyCredential azureKeyCredential = new AzureKeyCredential(apiKey);
OpenAIClient client = new OpenAIClient(
    apiEndpoint,
    azureKeyCredential
);
Console.WriteLine($"Azure OpenAI client created!");

//Establish Redis Connection
ConnectionMultiplexer redisConnection = ConnectionMultiplexer.Connect(redisUri);
IDatabase redisDatabase = redisConnection.GetDatabase();
Console.WriteLine("Connection to Redis created!");

#!share --from c# redisDatabase --as redisDatabase

Installed Packages Azure.AI.OpenAi, 1.0.0-beta.5 NRedisStack, 0.6.1

Azure OpenAI client created!
Connection to Redis created!


### Step 4: Embeddings

Usage of Azure OpenAI's `text-embedding-ada-002` model to created embeddings.


In [7]:
string modelDeploymentName = "<<your model deployment name>>";

EmbeddingsOptions embeddingsOptions; 
Dictionary<string, float[]> vectors = new Dictionary<string, float[]>(); 
foreach (var fact in facts) {
    embeddingsOptions = new EmbeddingsOptions(fact.Value){
        InputType = fact.Key
    };
    Response<Embeddings> embedding = await client.GetEmbeddingsAsync(modelDeploymentName, embeddingsOptions); 
    vectors.Add(fact.Key, embedding.Value.Data[0].Embedding.ToArray<float>()); 
    Console.WriteLine($"Embedding for {fact.Key} created!");
}

#!share --from c# modelDeploymentName --as modelDeploymentName

Embedding for Mercury created!
Embedding for Saturn created!
Embedding for Germany created!


### Step 5: Store vectors

The calculated vectors are stored in Redis using HashSets

In [8]:
//Store Embeddings/Vectors in Redis
foreach (var vector in vectors)
{
    redisDatabase.HashSet($"vec:{vector.Key}", new HashEntry[]
    {
        new("vector", (vector.Value).SelectMany(BitConverter.GetBytes).ToArray()),
        new("key", $"{vector.Key}")
    });
    Console.WriteLine($"Embedding for {vector.Key} stored in Redis");
}

Embedding for Mercury stored in Redis
Embedding for Saturn stored in Redis
Embedding for Germany stored in Redis


### Step 6: Create Search Index

Creation of a search index on the vectors to query them.

In [9]:
//Create Redis Search Index
ISearchCommands ft = redisDatabase.FT();
try {ft.DropIndex("search_index");} catch {};  //delete index in case it already exists

Dictionary<string, object> vectorAttributes = new Dictionary<string, object>(); 
vectorAttributes.Add("DIM", 1536);
vectorAttributes.Add("TYPE", "FLOAT32");
vectorAttributes.Add("DISTANCE_METRIC", "L2");

ft.Create("search_index", 
    new FTCreateParams().On(IndexDataType.HASH).Prefix("vec:"),
    new Schema().AddTagField("key").AddVectorField("vector", VectorField.VectorAlgo.FLAT, vectorAttributes)); 

Console.WriteLine("Redis search index created!"); 

Redis search index created!


### Step 7: Search

Redis Stack allows based on the previously created search index a semantic search. In the example a vector representation (embedding) of the query **Name planets in our solar system** will be created and used to perform a semantic search. Results of the semantic search, including the score are displayed.

In [10]:
//Perform semantic search
string query = "Name planets in our solar system"; 
embeddingsOptions = new EmbeddingsOptions(query){
    InputType = "query"
};
Response<Embeddings> embeddings = await client.GetEmbeddingsAsync(modelDeploymentName, embeddingsOptions); 
float[] searchVector = embeddings.Value.Data[0].Embedding.ToArray<float>();
SearchResult searchResult = ft.Search("search_index",
    new Query("*=>[KNN 2 @vector $searchVector]")
    .AddParam("searchVector", searchVector.SelectMany(BitConverter.GetBytes).ToArray())
    .SetSortBy("__vector_score")
    .Dialect(2));

foreach (Document doc in searchResult.Documents) {
    Console.WriteLine($"Document: {doc.Id}, Score: {doc.GetProperties().Where(x => x.Key == "__vector_score").FirstOrDefault().Value}");
}

Document: vec:Mercury, Score: 0.29868093133
Document: vec:Saturn, Score: 0.309674471617


## Summary

Redis and Redis Stack in combination with OpenAI can be used to calculate, store and semantically query existing data. This could be used provide primary content to prompts.